# Multi-Layer Perceptron Regressor

> for more information about MLP-regressor follow the [link](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html?highlight=multi%20layer%20perceptron%20regressor)

## Import libraries

In [ ]:
# reload modules before executing user code
%load_ext autoreload
# reload all modules every time before executing Python code
%autoreload 2
# render plots in notebook
%matplotlib inline

In [ ]:
# data wrangling
import pandas as pd
import numpy as np
from dslectures.core import *
from pathlib import Path

# data viz
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg

sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))

# ml magic
# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


from pandas.api.types import is_object_dtype, is_numeric_dtype

## Load the data

We also make use of the `pathlib` library to handle our filepaths:

In [ ]:
datapath = Path('../data')
%ls {datapath}

processed/ raw/


In [ ]:
train_data = pd.read_csv(datapath/'raw/train.csv'); train_data.head(); train_data.head().T

,0,1,2,3,4
artist_name,Netherfriends,Maxo Kream,Drebae,TWICE,Lisa Howard
track_id,7luDJV4DDZmjH3QDdCqpcO,1F09jtMfeYNkUj6piQjXwM,2tR2VS8rAndKb0C5hS3IpD,1U3cHXWaa0FqlUWLLBL7Kz,1vAP99gg1HzuUUITxJPdyn
track_name,Money Everyday,ATW,Trust in Me (feat. Allura),BRAND NEW GIRL,Cheeseburger In Paradise
acousticness,0.0722,0.155,0.239,0.016,0.281
danceability,0.79,0.677,0.661,0.674,0.607
duration_ms,186410,149967,231732,213956,228556
energy,0.384,0.747,0.625,0.965,0.771
instrumentalness,1.72e-06,0,0,9.59e-05,5.29e-06
key,7,6,6,7,2
liveness,0.1,0.239,0.0971,0.145,0.221


In [ ]:
test_data = pd.read_csv(datapath/'raw/test.csv'); test_data.head(); test_data.head(1).T

,0
artist_name,Supriya Lohith
track_id,7rrY55kdGxRC65rfeeJkG0
track_name,Nade Nade
acousticness,0.485
danceability,0.764
duration_ms,192000
energy,0.605
instrumentalness,1.19e-05
key,2
liveness,0.0915


---

#### Exercise #1

Even though you may be told a dataset has been cleaned and prepared for training a model, you should always perform some sanity checks! 

* Check that `housing_data` is free of missing values
* Check that all columns are numerical

---

In [ ]:
train_data.isnull().sum()

artist_name         0
track_id            0
track_name          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
popularity          0
collection_date     0
dtype: int64

In [ ]:
train_data.dtypes

artist_name          object
track_id             object
track_name           object
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
popularity            int64
collection_date      object
dtype: object

In [ ]:
def getting_rid_features(data):
    data.drop(columns=["artist_name", "track_id", "track_name","collection_date"],axis=1,inplace=True)
    return data

In [ ]:
def convert_strings_to_categories(data):
    for col in data.columns:
        if is_object_dtype(data[col]):
            data[col] = data[col].astype('category')

In [ ]:
train_data.to_csv(datapath/'processed/train_data_processed_schab9.csv', index=False)

## Select a performance measure

Before we can train any model, we need to think about which performance measure we wish to optimise for. For regression problems the Root Mean Square Error (RMSE) is often used as it measures the _**standard deviation**_ of the errors the algorithm makes in its predictions and gives a higher weight to large errors. For example, an RMSE equal to 50,000 means that about 68% of the algorithm's predictions fall within 50,000 CHF of the actual value, and about 95% fall within 100,000 CHF.

> Note: In general, lower values of RMSE indicate a better fit to the data.

Mathematically, the formula for RMSE is:

$$\mathrm{RMSE} = \sqrt{\frac{1}{m}\sum_{i=1}^m \left(\hat{y}_i - y_i\right)^2}$$

*One possibility to calculate the deviation*

where $m$ is the number of instances in the dataset you are measuring the RMSE on, $\hat{y}_i$ is the model's prediction for the $i^{th}$ instance, and $y_i$ is the actual label. Let's create a simple function that uses scitkit-learn's [mean_squared_error function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (which is just RMSE$^2$):

In [ ]:
def rmse(y, yhat):
    """A utility function to calculate the Root Mean Square Error (RMSE).
    
    Args:
        y (array): Actual values for target.
        yhat (array): Predicted values for target.
        
    Returns:
        rmse (double): The RMSE.
    """
    return np.sqrt(mean_squared_error(y, yhat))

### The Multi-Layer Perceptron Regressor


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.base import *

In [ ]:
getting_rid_features(train_data)
train_data.head().T

,0,1,2,3,4
acousticness,0.072200,0.155,0.2390,0.016000,0.281000
danceability,0.790000,0.677,0.6610,0.674000,0.607000
duration_ms,186410.000000,149967.000,231732.0000,213956.000000,228556.000000
energy,0.384000,0.747,0.6250,0.965000,0.771000
instrumentalness,0.000002,0.000,0.0000,0.000096,0.000005
key,7.000000,6.000,6.0000,7.000000,2.000000
liveness,0.100000,0.239,0.0971,0.145000,0.221000
loudness,-11.201000,-5.680,-8.0900,-3.754000,-5.106000
mode,1.000000,0.000,0.0000,1.000000,1.000000
speechiness,0.231000,0.318,0.1010,0.069600,0.112000


### Arrange data into a feature matrix and target vector

In [ ]:
train_data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0.0722,0.790,186410,0.384,0.000002,7,0.1000,-11.201,1,0.2310,156.077,4,0.203,2
1,0.1550,0.677,149967,0.747,0.000000,6,0.2390,-5.680,0,0.3180,153.921,4,0.371,41
2,0.2390,0.661,231732,0.625,0.000000,6,0.0971,-8.090,0,0.1010,132.899,4,0.385,9
3,0.0160,0.674,213956,0.965,0.000096,7,0.1450,-3.754,1,0.0696,157.041,4,0.750,41
4,0.2810,0.607,228556,0.771,0.000005,2,0.2210,-5.106,1,0.1120,138.087,4,0.496,19


In [ ]:
train_data.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
count,117293.000000,117293.000000,1.172930e+05,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000,117293.000000
mean,0.342946,0.581336,2.125350e+05,0.568661,0.224272,5.232247,0.194912,-9.987972,0.608101,0.112156,119.464675,3.878654,0.439534,24.152592
std,0.345778,0.190210,1.226774e+05,0.260323,0.360387,3.602673,0.167941,6.549731,0.488176,0.124630,30.154314,0.514728,0.259093,19.664392
min,0.000000,0.000000,3.203000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.031600,0.459000,1.638140e+05,0.395000,0.000000,2.000000,0.097500,-11.923000,0.000000,0.038900,96.014000,4.000000,0.223000,7.000000
50%,0.204000,0.605000,2.018130e+05,0.602000,0.000149,5.000000,0.124000,-7.989000,1.000000,0.055900,120.025000,4.000000,0.420000,21.000000
75%,0.638000,0.727000,2.410130e+05,0.775000,0.444000,8.000000,0.236000,-5.690000,1.000000,0.129000,139.646000,4.000000,0.638000,38.000000
max,0.996000,0.996000,5.610020e+06,1.000000,1.000000,11.000000,0.999000,1.806000,1.000000,0.966000,249.983000,5.000000,1.000000,100.000000


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117293 entries, 0 to 117292
Data columns (total 14 columns):
acousticness        117293 non-null float64
danceability        117293 non-null float64
duration_ms         117293 non-null int64
energy              117293 non-null float64
instrumentalness    117293 non-null float64
key                 117293 non-null int64
liveness            117293 non-null float64
loudness            117293 non-null float64
mode                117293 non-null int64
speechiness         117293 non-null float64
tempo               117293 non-null float64
time_signature      117293 non-null int64
valence             117293 non-null float64
popularity          117293 non-null int64
dtypes: float64(9), int64(5)
memory usage: 12.5 MB


In [ ]:
y = train_data['popularity']
X = train_data.drop(columns='popularity')


In [ ]:
#X, y = make_regression(n_samples=10000, random_state=1)#n_samples default = 100
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)
print(f' We have {len(X_train)} train rows + {len(X_valid)} valid rows')
model = MLPRegressor(random_state=1, max_iter=500)
model.fit(X_train, y_train)
#y_pred_test = model.predict(test_data[:2])
#y_pred_test
model.predict(X_valid[:2])

 We have 93834 train rows + 23459 valid rows


array([ 8.41427276, 19.03573542])

In [ ]:
X.shape

(117293, 13)

model.get_params returns a dictonary of all parameters for this estimators

In [ ]:
model.get_params(deep=True)

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

### Evaluate the predictions  

model.score() returns the coefficient of determination $R^2$ of the prediction.  

The coefficient $R^2$ is defined as $(1-(u/v))$

$u$ is the residual sum of squares $\sum(y_{true} - y_{pred})^2$  

ans $v$ is the total sum of squares $\sum(y_{true} - \bar{y}_{true})^2$  

the best possible score is 1.0 for $R^2$

In [ ]:
model.score(X_valid, y_valid)

-0.299125312513981

In [ ]:
train_data.shape

(117293, 14)

In [ ]:
data_merged = train_data.append(test_data)

/Users/leandro/git/dslectures/env/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [ ]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130326 entries, 0 to 13032
Data columns (total 18 columns):
acousticness        130326 non-null float64
artist_name         13033 non-null object
collection_date     13033 non-null object
danceability        130326 non-null float64
duration_ms         130326 non-null int64
energy              130326 non-null float64
instrumentalness    130326 non-null float64
key                 130326 non-null int64
liveness            130326 non-null float64
loudness            130326 non-null float64
mode                130326 non-null int64
popularity          117293 non-null float64
speechiness         130326 non-null float64
tempo               130326 non-null float64
time_signature      130326 non-null int64
track_id            13033 non-null object
track_name          13033 non-null object
valence             130326 non-null float64
dtypes: float64(10), int64(4), object(4)
memory usage: 18.9+ MB


In [ ]:
data_merged.shape

(130326, 18)

In [ ]:
y = data_merged['popularity']
X = data_merged.drop(columns='popularity')

In [ ]:
#X, y = make_regression(n_samples=1000, random_state=1)#n_samples default = 100
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)
print(f' We have {len(X_train)} train rows + {len(X_valid)} valid rows')
model = MLPRegressor(random_state=1, max_iter=500)
model.fit(X_train, y_train)
#y_pred_test = model.predict(test_data[:2])
#y_pred_test
model.predict(X[:2])

 We have 104260 train rows + 26066 valid rows


ValueError: could not convert string to float: "The O'Neill Brothers Group"

In [ ]:
data_merged[117294:130326]

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
1,0.99500,0.234,131000,0.0637,0.91800,7,0.2120,-26.841,0,0.0433,70.251,4,0.2010,NaN
2,0.78200,0.591,192697,0.5580,0.00000,5,0.6500,-6.492,0,0.1200,79.961,4,0.3970,NaN
3,0.07290,0.660,215381,0.6480,0.00000,2,0.1920,-9.691,1,0.0480,129.045,4,0.8360,NaN
4,0.98600,0.472,494418,0.1140,0.53400,3,0.1030,-20.791,1,0.0613,69.240,3,0.0345,NaN
5,0.00155,0.831,239518,0.5870,0.01810,1,0.0855,-7.439,1,0.1150,125.991,4,0.7860,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13028,0.05220,0.766,144000,0.5760,0.00000,7,0.1130,-9.302,1,0.0719,149.955,4,0.6860,NaN
13029,0.99400,0.432,147347,0.1440,0.89300,10,0.1040,-25.868,1,0.0389,106.833,4,0.8860,NaN
13030,0.92600,0.304,102582,0.1850,0.92300,3,0.1570,-31.639,0,0.0467,172.967,4,0.8850,NaN
13031,0.02160,0.632,200250,0.4980,0.00000,10,0.1130,-6.136,0,0.0377,160.224,4,0.5320,NaN


In [ ]:
y_pred_test = model.predict(test_data[:2])

ValueError: X has 13 features, but MLPRegressor is expecting 100 features as input.

In [ ]:
submission = pd.read_csv(datapath/'submitted/sample_submission.csv')
submission.shape

In [ ]:
submission['popularity'] = y_pred_test
submission.to_csv(datapath/'submitted/first_submission_schab9.csv', index=False)